# Keras Tuner

### It helps to know that Howmany  CNN layers are require for  the model .  

In [1]:
!pip install keras-tuner


     |████████████████████████████████| 102kB 6.2MB/s 
  Created wheel for kt-legacy: filename=kt_legacy-1.0.3-cp37-none-any.whl size=9569 sha256=5d9939028de3b473d04775de2557a8e34ca077b19553b4ee083721970170f70d
  Stored in directory: /root/.cache/pip/wheels/95/99/a8/6b116b0f69be60cc475d1cd36680f3a09f284d86655bb99d93
Successfully built kt-legacy


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
fation_mnist = keras.datasets.fashion_mnist

In [4]:
(X_train,y_train),(X_test,y_test) = fation_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


### Scaling the image data

In [5]:
X_train = X_train/255.0
X_test = X_test/255.0

In [6]:
X_train[0].shape

(28, 28)

In [7]:
###  Reshape the image

X_train = X_train.reshape(len(X_train),28,28,1)
X_test = X_test.reshape(len(X_test),28,28,1)

In [8]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [9]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [10]:
tunner_search = RandomSearch(build_model,
                             objective='val_accuracy',
                             max_trials=5,
                             directory='output',
                             project_name='Emnist Fation')

In [11]:
tunner_search.search  (X_train,y_train,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 30s]
val_accuracy: 0.875166654586792

Best val_accuracy So Far: 0.8796666860580444
Total elapsed time: 00h 03m 40s
INFO:tensorflow:Oracle triggered exit


In [12]:
model = tunner_search.get_best_models(num_models=1)[0]

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 80)        800       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        128064    
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2973792   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 3,103,626
Trainable params: 3,103,626
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(X_train,y_train,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2701 - accuracy: 0.9012 - val_loss: 0.3812 - val_accuracy: 0.8732
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2447 - accuracy: 0.9118 - val_loss: 0.3924 - val_accuracy: 0.8732
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2369 - accuracy: 0.9152 - val_loss: 0.3983 - val_accuracy: 0.8752
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2183 - accuracy: 0.9226 - val_loss: 0.5082 - val_accuracy: 0.8735
Epoch 8/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2008 - accuracy: 0.9297 - val_loss: 0.4733 - val_accuracy: 0.8712
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2041 - accuracy: 0.9305 - val_loss: 0.4947 - val_accuracy: 0.8723
Epoch 10/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1815 - accuracy: 0.9369 - val_loss: 0.5233 - val_accuracy